# 3. Multi-class Vnet on BV

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from dataset import *
from vnet import *
from training import *
from niiutility import show_image, show_batch_image

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 3.1 Setup Torch Global Variable, load memory map 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, sampler, SubsetRandomSampler
from torchvision import transforms, utils

import torch.nn.functional as F  # useful stateless functions
import torchvision.transforms as T

#------------------------------- GLOBAL VARIABLES -------------------------------------#

USE_GPU = True
BATCH_SIZE = 3
NUM_WORKERS = 6
NUM_TRAIN = 72
LEARNING_RATE = 1e-2

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    print('using GPU for training')
else:
    device = torch.device('cpu')

using GPU for training


In [4]:
#-------------------------LOAD THE DATA SET-------------------------------------------#

data_index = np.arange(107)
data_index = np.delete(data_index, 46)
dataset_trans = BvMaskDataset(data_index, 
                         transform=transforms.Compose([
                             downSample(2),
                             RandomFilp(0.5),
                             RandomAffine(90, 5)
                         ])
                     )

#-------------------------CREATE DATA LOADER FOR TRAIN AND VAL------------------------#

data_size = len(dataset_trans)
train_loader = DataLoader(dataset_trans, batch_size=BATCH_SIZE, \
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)),\
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(dataset_trans, batch_size=BATCH_SIZE,
                    sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN,data_size)),\
                    num_workers=NUM_WORKERS)

* Print first 4 batch of data

In [6]:
from rpn import RPN3D
from vnet import LNet
#-------------------------NEW MODEL INIT WEIGHT--------------------------------------#

model = LNet(img_size=(64, 96, 96), out_size=6)
model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
#optimizer = optim.SGD (model.parameters(),lr=LEARNING_RATE, momentum=0.99)

from loss import *


In [ ]:
train(model, train_loader, validation_loader, optimizer,\
      device=device, dtype=dtype, lossFun=IoU, epochs=1500)

epoch 0 begins: 
Epoch 0 finished ! Training Loss: 1.5810050342393958
     validation loss = 1.4466
epoch 1 begins: 
Epoch 1 finished ! Training Loss: 0.5999755677969559
     validation loss = 0.9187
epoch 2 begins: 
Epoch 2 finished ! Training Loss: 0.5256082117557526
     validation loss = 0.5992
epoch 3 begins: 
Epoch 3 finished ! Training Loss: 0.5073198676109314
     validation loss = 4.0348
epoch 4 begins: 
Epoch 4 finished ! Training Loss: 0.44026054117990576
     validation loss = 0.1346
epoch 5 begins: 
Epoch 5 finished ! Training Loss: 0.35564502296240436
     validation loss = 0.4490
epoch 6 begins: 
Epoch 6 finished ! Training Loss: 0.4735383961511695
     validation loss = 0.4652
epoch 7 begins: 
Epoch 7 finished ! Training Loss: 0.43970636829085974
     validation loss = 0.5079
epoch 8 begins: 
Epoch 8 finished ! Training Loss: 0.4468813344188359
     validation loss = 0.4513
epoch 9 begins: 
Epoch 9 finished ! Training Loss: 0.4303887305052384
     validation loss = 0.55

In [ ]:
#-------------------------SAVE THE MODEL STATE DICT----------------------------------#
PATH = 'Vet_Mask_class_2'
torch.save(model.state_dict(), PATH)